In [ ]:
# ============================================================================
# FEATURE ENGINEERING
# ============================================================================

# Import Libraries


# Load Data


# Create Derived Features (rank difference, age difference, etc.)


# Create Surface-Specific Win Rates


# Create Recent Form Features (last 5/10 matches)


# Create Head-to-Head Features


# (Optional) Create ELO Ratings


# Save Enhanced Dataset